In [1]:
MASTERNODE_PRIVATE_IP="192.168.2.30"

In [2]:
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession.builder\
        .master(f"spark://{MASTERNODE_PRIVATE_IP}:7077") \
        .appName("Lecture2_Example1_Shared_State")\
        .config("spark.executor.cores",4)\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled", True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()


# Old API (RDD)
spark_context = spark_session.sparkContext
spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/16 08:38:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/16 08:38:04 WARN Utils: Service 'sparkDriver' could not bind on port 9999. Attempting port 10000.
26/02/16 08:38:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/02/16 08:38:06 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 10005. Attempting port 10006.


---
## 1. Broadcast Variables

In [3]:
# Immutable data for all workers
broadcast_variable = spark_context.broadcast({"apples": 100, "oranges": 42, "pears": 1})

In [4]:
rdd = spark_context.parallelize(["apples", "pears"])

# Convert fruit to amounts of fruit.
# "value" is captured in the closure, but its only a handle - the dictionary is serialized via a different route
result = rdd.map(lambda x: broadcast_variable.value[x])\
            .collect()

print(result)

[100, 1]


## 2. Accumulator Variables

In [5]:
# ACCUMULATOR 
accumulator_1 = spark_context.accumulator(0)
rdd2 = spark_context.parallelize([1,2,3,4,5,6], 2)

In [6]:
result = rdd2.foreach(lambda x: accumulator_1.add(1))

print(accumulator_1.value)

6


In [7]:
# release the cores for another application!
spark_context.stop()